<a href="https://colab.research.google.com/github/Shree-crypto/YingYang/blob/master/genji_python_6b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Genji is a transformer model finetuned on EleutherAI's GPT-J 6B model. This particular model is trained on python only code approaching 4GB in size.

Finetuned by NovelAI, massive thanks to TPU Research Cloud for providing hardware to make this project possible and thanks to EleutherAI for pretraining GPT-J 6B. 

Check the model on Huggingface for more details: [HuggingFace link](https://huggingface.co/NovelAI/genji-python-6B)

In [ ]:
#@title Install the requirements
#INSTALL THE REQUIREMENTS
!apt install git-lfs
!pip install git+https://github.com/finetuneanon/transformers@gpt-neo-localattention3-rp-b
#import deps
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    GPTNeoForCausalLM,
)
import torch
import psutil
low_ram=False
ram_stats = psutil.virtual_memory()
if ram_stats.available < 20 * 1000 * 1000 * 1000:
  low_ram=True
print(low_ram)

In [ ]:
#@title Load the model(Might take a while!)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B")
if low_ram:
  !git lfs install
  !git clone https://huggingface.co/NovelAI/genji-python-6B-split
  model = AutoModelForCausalLM.from_pretrained("/content/genji-python-6B-split/model").half().eval().cuda()
else:
  model = AutoModelForCausalLM.from_pretrained("NovelAI/genji-python-6B").half().eval().cuda()

In [ ]:
#@title Set up the generator
maxLength=200#@param {type:"number"}
temperature=0.4#@param {type:"number"}
top_k = 50#@param {type:"number"}
top_p = 0.9#@param {type:"number"}
repetition_penalty = 1.13#@param {type:"number"}
repetition_penalty_range = 512#@param {type:"number"}
repetition_penalty_slope = 3.33#@param {type:"number"}
def generator(text):
    tokens = tokenizer(text, return_tensors="pt").input_ids.cuda()[:, -(2047-maxLength):]
    out = model.generate(
        tokens.long(),
        do_sample=True,
        min_length=tokens.shape[1] + maxLength,
        max_length=tokens.shape[1] + maxLength,
        temperature=temperature,
        top_k = top_k,
        top_p = top_p,
        repetition_penalty = repetition_penalty,
        repetition_penalty_range = repetition_penalty_range,
        repetition_penalty_slope = repetition_penalty_slope,
        use_cache=True,
        bad_words_ids=None,
        pad_token_id=tokenizer.eos_token_id,
    ).long().to("cpu")[0]
    return tokenizer.decode(out[tokens.shape[1]:])

In [ ]:
#@title Launch the editor to generate
from ipywidgets import interact, widgets
from IPython.display import display
text = widgets.Textarea(
    value='"""\nThis function calculates the area of a circle given it\'s radius.\n"""\ndef',
    placeholder='Define your prompt here',
    disabled=False,
    layout={'width': '100%', 'height': '300px'},
)
display(text)

button = widgets.Button(description='Generate')

def clicked(b):
    button.disabled = True
    text.value = text.value + generator(text.value)
    button.disabled = False

button.on_click(clicked)

display(button)